待优化的函数：

In [1]:
import numpy as np


def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(
            ['mean', 'count'])
        result[i] = groupby_result.loc[
            groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result


In [2]:
import pandas as pd
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
result_1 = target_mean_v1(data, 'y', 'x')
%timeit target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 23.9 s per loop


第一次优化将时间复杂度由N2->N1

In [3]:
from collections import defaultdict

def target_mean_v2(data, y_name, x_name):
    count = data.shape[0]
    result = np.zeros(count)
    sum_dict = defaultdict(int)
    count_dict = defaultdict(int)
    for i in range(count):
        sum_dict[data.loc[i, x_name]] += data.loc[i, y_name]
        count_dict[data.loc[i, x_name]] += 1

    for i in range(count):
        result[i] = (sum_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (
                count_dict[data.loc[i, x_name]] - 1)
    return result

In [4]:
result_2 = target_mean_v2(data, 'y', 'x')

diff = np.linalg.norm(result_1 - result_2)
print(diff)

%timeit target_mean_v2(data, 'y', 'x')

0.0
1 loop, best of 3: 222 ms per loop


引入profiler查看代码运行效率情况 

第一次使用需要pip安装

In [5]:
pip install line_profiler

     |████████████████████████████████| 71kB 3.5MB/s 


一般来说写法是这样的

In [6]:
from line_profiler import LineProfiler

profiler = LineProfiler(target_mean_v2)  #把函数传递到性能分析器
profiler.enable()  #开始分析
target_mean_v2(data, 'y', 'x')  #开始执行
profiler.disable()  #停止分析
profiler.print_stats()  #打印出性能分析结果

Timer unit: 1e-06 s

Total time: 0.61772 s
File: <ipython-input-3-2636a9055c27>
Function: target_mean_v2 at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def target_mean_v2(data, y_name, x_name):
     4         1        137.0    137.0      0.0      count = data.shape[0]
     5         1        253.0    253.0      0.0      result = np.zeros(count)
     6         1          4.0      4.0      0.0      sum_dict = defaultdict(int)
     7         1          1.0      1.0      0.0      count_dict = defaultdict(int)
     8      5001       2781.0      0.6      0.5      for i in range(count):
     9      5000     199396.0     39.9     32.3          sum_dict[data.loc[i, x_name]] += data.loc[i, y_name]
    10      5000     103032.0     20.6     16.7          count_dict[data.loc[i, x_name]] += 1
    11                                           
    12      5001       2626.0      0.5      0.4      for i in range(count):
    13 

不过在colab可以这样玩

In [8]:
%load_ext line_profiler
%lprun -f target_mean_v2 target_mean_v2(data, 'y', 'x')

打印信息相同

Timer unit: 1e-06 s

Total time: 0.615588 s
File: <ipython-input-3-2636a9055c27>
Function: target_mean_v2 at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def target_mean_v2(data, y_name, x_name):
     4         1         74.0     74.0      0.0      count = data.shape[0]
     5         1        129.0    129.0      0.0      result = np.zeros(count)
     6         1          3.0      3.0      0.0      sum_dict = defaultdict(int)
     7         1          1.0      1.0      0.0      count_dict = defaultdict(int)
     8      5001       2572.0      0.5      0.4      for i in range(count):
     9      5000     203881.0     40.8     33.1          sum_dict[data.loc[i, x_name]] += data.loc[i, y_name]
    10      5000     106111.0     21.2     17.2          count_dict[data.loc[i, x_name]] += 1
    11                                           
    12      5001       2613.0      0.5      0.4      for i in range(count):
    13      5000     197596.0     39.5     32.1          result[i] = (sum_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (
    14      5000     102608.0     20.5     16.7                  count_dict[data.loc[i, x_name]] - 1)
    15         1          0.0      0.0      0.0      return result

由上面可见，更多的时间花费在9 10 13 14,但是由于那几行的语句比较复杂，无法定位hotspot，所以对代码进行解构，在进行分析

In [10]:
def target_mean_v2_decode(data, y_name, x_name):
    count = data.shape[0]
    result = np.zeros(count)
    sum_dict = defaultdict(int)
    count_dict = defaultdict(int)
    for i in range(count):
        sum_dict[data.loc[i, x_name]] += data.loc[i, y_name]
        temp_1 = data.loc[i, x_name]
        count_dict[temp_1] += 1

    for i in range(count):
        temp_2 = data.loc[i, x_name]
        temp_3 = data.loc[i, y_name]
        result[i] = (sum_dict[temp_2] - temp_3) / (count_dict[temp_2] - 1)
    return result

In [12]:
%lprun -f target_mean_v2_decode target_mean_v2_decode(data, 'y', 'x')

Timer unit: 1e-06 s

Total time: 0.538931 s
File: <ipython-input-10-38d12b65efab>
Function: target_mean_v2_decode at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def target_mean_v2_decode(data, y_name, x_name):
     2         1         24.0     24.0      0.0      count = data.shape[0]
     3         1         83.0     83.0      0.0      result = np.zeros(count)
     4         1          2.0      2.0      0.0      sum_dict = defaultdict(int)
     5         1          0.0      0.0      0.0      count_dict = defaultdict(int)
     6      5001       2854.0      0.6      0.5      for i in range(count):
     7      5000     209125.0     41.8     38.8          sum_dict[data.loc[i, x_name]] += data.loc[i, y_name]
     8      5000     103627.0     20.7     19.2          temp_1 = data.loc[i, x_name]
     9      5000       5248.0      1.0      1.0          count_dict[temp_1] += 1
    10                                           
    11      5001       2736.0      0.5      0.5      for i in range(count):
    12      5000     104344.0     20.9     19.4          temp_2 = data.loc[i, x_name]
    13      5000     102273.0     20.5     19.0          temp_3 = data.loc[i, y_name]
    14      5000       8615.0      1.7      1.6          result[i] = (sum_dict[temp_2] - temp_3) / (count_dict[temp_2] - 1)
    15         1          0.0      0.0      0.0      return result

不出所料时间大多花费在7 8 12 13行，他们都是panda里定位数据的loc方法。我们考虑用该方法外把数据以列表的形式一次提取出来，提供给函数使用

In [13]:
def target_mean_v3(y_values, x_values):
    sum_dict = defaultdict(int)
    count_dict = defaultdict(int)

    for x_v, y_v in zip(x_values, y_values):
        sum_dict[x_v] += y_v
        count_dict[x_v] += 1

    result = [(sum_dict[x_v] - y_v)/(count_dict[x_v]-1) for x_v, y_v in zip(x_values, y_values)]
    return result

%lprun -f target_mean_v3 target_mean_v3(data['y'].values, data['x'].values)

Timer unit: 1e-06 s

Total time: 0.019567 s
File: <ipython-input-13-b7210d918f95>
Function: target_mean_v3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def target_mean_v3(y_values, x_values):
     2         1          4.0      4.0      0.0      sum_dict = defaultdict(int)
     3         1          2.0      2.0      0.0      count_dict = defaultdict(int)
     4                                           
     5      5001       4866.0      1.0     24.9      for x_v, y_v in zip(x_values, y_values):
     6      5000       4534.0      0.9     23.2          sum_dict[x_v] += y_v
     7      5000       4098.0      0.8     20.9          count_dict[x_v] += 1
     8                                           
     9         1       6062.0   6062.0     31.0      result = [(sum_dict[x_v] - y_v)/(count_dict[x_v]-1) for x_v, y_v in zip(x_values, y_values)]
    10         1          1.0      1.0      0.0      return result

问题得以解决，然后让我们看看优化后的结果是否正确，性能提升如何

In [14]:
result_3 = target_mean_v3(data['y'].values, data['x'].values)

diff = np.linalg.norm(result_1 - result_3)
print(diff)

%timeit target_mean_v3(data['y'].values, data['x'].values)

0.0
100 loops, best of 3: 6.92 ms per loop


相对于第一次的 23.9 s per loop， 6.92 ms per loop性能提升了大约3500倍


python层面的算法我们优化的差不多了，是时候进行底层的优化了

**Cython优化**

Q:为什么用Cython优化这么麻烦，为什么不直接用C来写？
A:因为会遇到数据传输的问题，numpy的一些操作只有在cython中实现


使用之前，我们要在colab里面加载cython

In [15]:
%load_ext Cython

然后在每个代码块前加上%%cython,需要cython优化的函数和变量用 cdef 来声明

-a可以显示编译之后的信息

In [84]:
%%cython -a
from collections import defaultdict
def target_mean_v4(y_values, x_values):
  cdef:
    object sum_dict = defaultdict(int)
    object count_dict = defaultdict(int)

  for x_v, y_v in zip(x_values, y_values):
      sum_dict[x_v] += y_v
      count_dict[x_v] += 1

  result = [(sum_dict[x_v] - y_v)/(count_dict[x_v]-1) for x_v, y_v in zip(x_values, y_values)]
  return result

In [85]:
%timeit target_mean_v4(data['y'].values, data['x'].values)

100 loops, best of 3: 5.6 ms per loop


cython里只有一些原生的c type，没有对应defaultdict的type,只能用object来声明， 但是优化的速度不是那么明显，所以我们不得不尝试把函数的一些高级用法替换掉


详细参考：https://stackoverflow.com/questions/41905891/to-what-degree-might-this-extension-type-be-inefficient-due-to-the-use-of-an-obj

In [82]:
%%cython -a
import numpy as np
cimport numpy as cnp

def target_mean_v5(data, x_name, y_name):
  cdef:
    int data_shape = data.shape[0]
    cnp.ndarray[cnp.int_t] y_values = data[y_name].values
    cnp.ndarray[cnp.int_t] x_values = data[x_name].values
    cnp.ndarray[cnp.float64_t] result = np.zeros(data_shape, dtype=np.float64)
    dict sum_dict = {}
    dict count_dict = {}

  for i in range(data_shape):
    x_v = x_values[i]
    y_v = y_values[i]
    if x_v in sum_dict:
      sum_dict[x_v] += y_v
      count_dict[x_v] += 1
    else:
      sum_dict[x_v] = y_v
      count_dict[x_v] = 1
  for i in range(data_shape):
    result[i] = (sum_dict[x_values[i]] - y_values[i])/(count_dict[x_values[i]]-1)
  return result

In [83]:
%timeit target_mean_v5(data, 'x', 'y')

1000 loops, best of 3: 1.04 ms per loop


**去掉了python的高级用法，返璞归真，换成cython，真鸡儿快呀！！！！**

但是我们不能再使用line_profiler去分析hotspot了，所以要在python算法层面上，少用高级用法，返璞归真的前提下，尽量优化！！！

**Cython 使用OpenMP来突破GIL锁的桎槁**

1.   python函数转换成cython的cpdef函数
2.   使用prange解开GIL锁
3.   cnp的链表type转换成指针

In [73]:
%%cython -a

import numpy as np
from cython.parallel import prange

cpdef target_mean_v6(data, x_name, y_name):  # cpdef
  cdef:
    int data_shape = data.shape[0]
    long[:,] y_values = data[y_name].values  #换成long type的指针
    long[:,] x_values = data[x_name].values
    double[:,] result = np.zeros(data_shape, dtype=np.float64)
    double[:,] sum_dict= np.zeros(data_shape, dtype=np.float64)
    double[:,] count_dict = np.zeros(data_shape, dtype=np.float64)
    int i = 0

  for i in prange(data_shape, nogil=True):  #不使用gil锁
    sum_dict[x_values[i]] += y_values[i]
    count_dict[x_values[i]] += 1
  for i in prange(data_shape, nogil=True):
    result[i] = (sum_dict[x_values[i]] - y_values[i])/(count_dict[x_values[i]]-1)
  return result

In [74]:
%timeit target_mean_v6(data, 'x', 'y')

The slowest run took 6.73 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 65.3 µs per loop


壮哉！！ 没有桎槁的速度！！

接下来不用指针， 直接复用使用python已经分配出来的内存 memoryview

In [75]:
%%cython -a

import numpy as np
from cython.parallel import prange

cpdef target_mean_v7(data, x_name, y_name):
  cdef:
    int data_shape = data.shape[0]
    long[::1] y_values = np.asfortranarray(data[y_name].values) #[::1]    np.asfortranarry
    long[::1] x_values = np.asfortranarray(data[x_name].values)
    double[::1] result = np.zeros(data_shape, dtype=np.float64)
    double[::1] sum_dict= np.zeros(data_shape, dtype=np.float64)
    double[::1] count_dict = np.zeros(data_shape, dtype=np.float64)
    int i = 0

  for i in prange(data_shape, nogil=True): 
    sum_dict[x_values[i]] += y_values[i]
    count_dict[x_values[i]] += 1
  for i in prange(data_shape, nogil=True):
    result[i] = (sum_dict[x_values[i]] - y_values[i])/(count_dict[x_values[i]]-1)
  return result

In [76]:
%timeit target_mean_v7(data, 'x', 'y')

The slowest run took 17.87 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 70.1 µs per loop


效果一般
接下来关闭 越界检查  和 包裹

In [81]:
%%cython -a

import numpy as np
import cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v8(data, x_name, y_name):
  cdef:
    int data_shape = data.shape[0]
    long[::1] y_values = np.asfortranarray(data[y_name].values) #[::1]    np.asfortranarry
    long[::1] x_values = np.asfortranarray(data[x_name].values)
    double[::1] result = np.zeros(data_shape, dtype=np.float64)
    double[::1] sum_dict= np.zeros(data_shape, dtype=np.float64)
    double[::1] count_dict = np.zeros(data_shape, dtype=np.float64)
    int i = 0

  for i in prange(data_shape, nogil=True): 
    sum_dict[x_values[i]] += y_values[i]
    count_dict[x_values[i]] += 1
  for i in prange(data_shape, nogil=True):
    result[i] = (sum_dict[x_values[i]] - y_values[i])/(count_dict[x_values[i]]-1)
  return result

In [80]:
%timeit target_mean_v8(data, 'x', 'y')

The slowest run took 13.22 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 45.9 µs per loop
